In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher
import deepmatcher  as dm 
import torch
torch.cuda.is_available()
import json
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 296kB 11.0MB/s 
  ERROR: Failed building wheel for fasttextmirror
    Running setup.py install for fasttextmirror ... done


In [ ]:
dfs = []
for line in open(r'/content/drive/My Drive/computers_train_xlarge.json', 'r'):
        x = pd.DataFrame(json.loads(line),index=[0])
        dfs.append(x)
                   
df = pd.concat(dfs)

In [ ]:
df1 = df.loc[df['label']==1]
df2 = df.loc[df['label']==0][:len(df1)]

In [ ]:
df = pd.concat([df1,df2])

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
new_columns=[]
for i in df.columns:
  try:
    n = i.split('_')
    if n[1] != 'id':
      n = n[1]+'_'+n[0]
    else:
      n= n[2]+'_'+n[1]+'_'+n[0]
    print(n)
  except:
    n = i
  new_columns.append(n)

df.columns = new_columns

left_id
left_category
left_id_cluster
right_id
right_category
right_id_cluster
left_brand
right_brand
left_description
right_description
left_keyValuePairs
right_keyValuePairs
left_price
right_price
left_specTableContent
right_specTableContent
left_title
right_title


In [ ]:
columns_to_drop=['left_id_cluster','right_id_cluster','left_keyValuePairs', 'right_keyValuePairs']
df=df.drop(columns_to_drop,axis=1)

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=1)

train, validation = train_test_split(train, test_size=0.25, random_state=1)

del df

In [ ]:
train = train.reset_index().drop('index',axis=1)
test= test.reset_index().drop('index',axis=1)
validation= validation.reset_index().drop('index',axis=1)

In [ ]:
train.to_csv(r'/content/drive/My Drive/DeepMatcherTest/train.csv',index=False)
test.to_csv(r'/content/drive/My Drive/DeepMatcherTest/test.csv',index=False)
validation.to_csv(r'/content/drive/My Drive/DeepMatcherTest/validation.csv',index=False)

In [ ]:
train, validation, test = dm.data.process(
    path=r'/content/drive/My Drive/DeepMatcherTest',
    train='train.csv',
    validation='validation.csv',
    test='test.csv',
    id_attr='pair_id',
    label_attr='label',
    ignore_columns=('left_id','right_id'),embeddings='glove.6B.50d'
    )


Reading and processing data from "/content/drive/My Drive/DeepMatcherTest/train.csv"
0% [######                        ] 100% | ETA: 00:01:10
Reading and processing data from "/content/drive/My Drive/DeepMatcherTest/validation.csv"
0% [#######                       ] 100% | ETA: 00:00:18
Reading and processing data from "/content/drive/My Drive/DeepMatcherTest/test.csv"
/root/.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                           
 99%|█████████▉| 396469/400000 [00:09<00:00, 41023.72it/s]
Building vocabulary
0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17

Computing principal components
0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:04:20
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning:

In [ ]:
train_table = train.get_raw_table()
train_table.head()

,left_category,right_category,label,pair_id,left_brand,right_brand,left_description,right_description,left_price,right_price,left_specTableContent,right_specTableContent,left_title,right_title
0,computers_and_accessories,computers_and_accessories,0,8837825#16995189,,`` samsung '' @ en,"`` disco solido ssd , 500gb de capacidad , 2.5...",`` samsung 's unique and innovative 3d v-nand ...,,,,,`` samsung 850 evo 500gb mz-75e500b/eu '' @ es,`` samsung 850 evo series m.2 1tb sata 6gbps s...
1,computers_and_accessories,computers_and_accessories,0,16272671#4537309,,`` corsair '' @ en,,"`` ddr4 , 2400mhz , cl14 , 1.2v , xmp 2.0 , li...",,,categorie geheugen intern merk corsair product...,memory type ddr4 ( pc4-19200 ) capacity 32gb (...,`` corsair vengeance lpx cmk64gx4m4a2666c16 - ...,`` corsair vengeance lpx black 32gb ( 4x8gb ) ...
2,computers_and_accessories,computers_and_accessories,1,9115026#15088537,,,,"`` disco duro 3.5 , 10tb capacidad , sata 6gbp...",,,,,`` 10tb gold sata 6 gb/s 7200 rpm 256mb 3.5in ...,`` wd gold 10tb wd101kryz '' @ es
3,computers_and_accessories,computers_and_accessories,1,17253303#15348141,`` hp enterprise '',`` hp enterprise '',`` description : storageworks externalpart num...,`` description : hp 15/30gb dlt tape drive ( d...,,,specifications : category proliant sub-categor...,specifications : category proliant sub-categor...,"`` 242469-001 hp storageworks external '' , ``...","`` hp 242469-001 15/30-gb ext scsi dlt '' , ``..."
4,computers_and_accessories,computers_and_accessories,1,11118257#2335315,,`` seagate '',`` disco duro interno hdd seagate 1tb 3.5 '',`` description:5 x seagate barracuda ( the pow...,,,,specifications : category seagate harddrive su...,`` disco duro interno hdd seagate 1tb '',`` st1000dm003 seagate 1-tb 7.2k 3.5 6g sata '...


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=5,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9763808
===>  TRAIN Epoch 1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:10:57


Finished Epoch 1 || Run Time:  535.9 | Load Time:  121.8 || F1:  69.02 | Prec:  53.45 | Rec:  97.37 || Ex/s:  17.68

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:03


Finished Epoch 1 || Run Time:   84.1 | Load Time:   39.9 || F1:  74.81 | Prec:  61.18 | Rec:  96.25 || Ex/s:  31.26

* Best F1: tensor(74.8080, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:10:57


Finished Epoch 10 || Run Time:  536.5 | Load Time:  121.9 || F1:  96.10 | Prec:  92.91 | Rec:  99.52 || Ex/s:  17.66

===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:04


Finished Epoch 10 || Run Time:   84.4 | Load Time:   40.0 || F1:  86.37 | Prec:  80.16 | Rec:  93.61 || Ex/s:  31.15

* Best F1: tensor(86.3689, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(86.3689, device='cuda:0')

In [ ]:
model.run_eval(test,batch_size=8, device='cpu', progress_style='bar', log_freq=5, sort_in_buckets=None)



===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:06:56


Finished Epoch 10 || Run Time:  406.6 | Load Time:    9.9 || F1:  85.09 | Prec:  77.44 | Rec:  94.42 || Ex/s:   9.31



tensor(85.0902)

In [ ]:
model.save_state(r'/content/drive/My Drive/DeepMatcherTest/finalmodel.pth')
model.save_state(r'/content/drive/My Drive/DeepMatcherTest/finalmodel.sav')